In [114]:
!pip install pip==24.0.0

DEPRECATION: pytorch-lightning 1.8.2 has a non-standard dependency specifier torch>=1.9.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [115]:
!pip install pytorch_lightning==1.8.2

DEPRECATION: pytorch-lightning 1.8.2 has a non-standard dependency specifier torch>=1.9.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [116]:
import pandas as pd
import numpy as np

import torch
from torch.utils.data import Dataset
import torch.nn as nn
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from google.colab import drive

In [117]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [118]:
%%time
path = "/content/drive/MyDrive/Punto parcial final corte 3/"
datos = pd.read_excel(path + "CIRCULA_23_TRANSF_S.xlsx", sheet_name=1)

CPU times: user 6min 41s, sys: 3.56 s, total: 6min 44s
Wall time: 6min 57s


In [119]:
datos.head(3)

,Biblioteca de Prestamo,Meson de Circulación,Cod Biblioteca origen Item,Biblioteca origen Item,Cod de Localizacion,Localización Item,Prestamos en Sala,Prestamos Externo,Barcode,MMS Id,...,Fecha de ingreso del Item,Categoría Socio,Identificador Socio,dewey,Dewey_1,Dewey3,Dewey4.1,Dewey4.1 - Copia,Dewey_2,TIPO MATERIAL
0,Biblioteca Luis Ángel Arango,Préstamo Externo Calle 12 - Solo socios,BLAA,Biblioteca Luis Ángel Arango,DC1,Depósito C1,0,1,29004006168206,991009934269707486,...,1997-12-30 00:00:00,Categoría Biblioteca Virtual,3900488,Co868.5 S45p1,2,868.5 S45p1,868,868.0,860,LIBRO
1,Biblioteca Luis Ángel Arango,Préstamo Externo Calle 12 - Solo socios,BLAA,Biblioteca Luis Ángel Arango,DD1,Depósito D1,0,1,29004005863906,991013769509707486,...,1997-02-24 00:00:00,Categoría Biblioteca Virtual,3900488,928.61 S45j1,0,928.61 S45j1,928,928.0,920,LIBRO
2,Biblioteca Luis Ángel Arango,Préstamo Externo Calle 12 - Solo socios,BLAA,Biblioteca Luis Ángel Arango,DD1,Depósito D1,0,1,29004007799355,991008897989707486,...,2000-05-22 00:00:00,Categoría Biblioteca Virtual,3900488,986.251 B82,0,986.251 B82,986,986.0,980,LIBRO


In [120]:
datos.isna().sum()

,0
Biblioteca de Prestamo,0
Meson de Circulación,0
Cod Biblioteca origen Item,0
Biblioteca origen Item,0
Cod de Localizacion,0
Localización Item,1731
Prestamos en Sala,0
Prestamos Externo,0
Barcode,0
MMS Id,0


In [121]:
# Eliminar filas que cumplan con alguna de las condiciones
datos = datos[
    (datos["Prestamos Externo"] != 0)]

In [122]:
conteo_libros = datos['Título'].value_counts()
datos = datos[datos['Título'].isin(conteo_libros[conteo_libros > 1].index)]

In [123]:
conteo_libros.unique()

array([497, 418, 373, 340, 307, 284, 282, 281, 278, 277, 276, 257, 251,
       231, 219, 215, 209, 205, 190, 185, 178, 177, 176, 173, 163, 157,
       155, 153, 151, 148, 146, 144, 142, 141, 138, 136, 135, 133, 132,
       131, 130, 129, 128, 127, 124, 123, 122, 120, 119, 118, 117, 116,
       115, 114, 113, 111, 110, 109, 108, 107, 106, 105, 104, 103, 102,
       101, 100,  99,  98,  97,  96,  95,  94,  93,  91,  90,  89,  88,
        87,  86,  85,  84,  83,  82,  81,  80,  79,  78,  77,  76,  75,
        74,  73,  72,  71,  70,  69,  68,  67,  66,  65,  64,  63,  62,
        61,  60,  59,  58,  57,  56,  55,  54,  53,  52,  51,  50,  49,
        48,  47,  46,  45,  44,  43,  42,  41,  40,  39,  38,  37,  36,
        35,  34,  33,  32,  31,  30,  29,  28,  27,  26,  25,  24,  23,
        22,  21,  20,  19,  18,  17,  16,  15,  14,  13,  12,  11,  10,
         9,   8,   7,   6,   5,   4,   3,   2,   1])

In [124]:
conteo_libros_socios = datos['Identificador Socio'].value_counts()
datos = datos[datos['Identificador Socio'].isin(conteo_libros_socios[conteo_libros_socios > 10].index)]

In [125]:
conteo_libros_socios.unique()

array([759, 546, 535, 489, 468, 417, 350, 323, 312, 235, 234, 230, 225,
       221, 202, 192, 190, 189, 185, 182, 177, 170, 165, 163, 162, 160,
       154, 152, 151, 146, 145, 144, 142, 141, 140, 139, 138, 137, 134,
       133, 130, 129, 128, 127, 125, 122, 120, 119, 118, 117, 116, 115,
       114, 113, 112, 111, 110, 109, 108, 106, 105, 104, 103, 102, 101,
       100,  99,  98,  97,  96,  95,  94,  93,  92,  91,  90,  89,  88,
        87,  86,  85,  84,  83,  82,  81,  80,  79,  78,  77,  76,  75,
        74,  73,  72,  71,  70,  69,  68,  67,  66,  65,  64,  63,  62,
        61,  60,  59,  58,  57,  56,  55,  54,  53,  52,  51,  50,  49,
        48,  47,  46,  45,  44,  43,  42,  41,  40,  39,  38,  37,  36,
        35,  34,  33,  32,  31,  30,  29,  28,  27,  26,  25,  24,  23,
        22,  21,  20,  19,  18,  17,  16,  15,  14,  13,  12,  11,  10,
         9,   8,   7,   6,   5,   4,   3,   2,   1])

In [152]:
datos = datos.drop_duplicates(subset=['Identificador Socio', 'Título'], keep='first')

In [153]:
datos.shape

(171467, 31)

In [154]:
datos.head(2)

,Biblioteca de Prestamo,Meson de Circulación,Cod Biblioteca origen Item,Biblioteca origen Item,Cod de Localizacion,Localización Item,Prestamos en Sala,Prestamos Externo,Barcode,MMS Id,...,Categoría Socio,Identificador Socio,dewey,Dewey_1,Dewey3,Dewey4.1,Dewey4.1 - Copia,Dewey_2,TIPO MATERIAL,rank_recientes
0,Biblioteca Luis Ángel Arango,Préstamo Externo Calle 12 - Solo socios,BLAA,Biblioteca Luis Ángel Arango,DC1,Depósito C1,0,1,29004006168206,991009934269707486,...,Categoría Biblioteca Virtual,4953,Co868.5 S45p1,2,868.5 S45p1,868,868.0,860,LIBRO,16.0
1,Biblioteca Luis Ángel Arango,Préstamo Externo Calle 12 - Solo socios,BLAA,Biblioteca Luis Ángel Arango,DD1,Depósito D1,0,1,29004005863906,991013769509707486,...,Categoría Biblioteca Virtual,4953,928.61 S45j1,0,928.61 S45j1,928,928.0,920,LIBRO,29.0


In [129]:
#%%time
# Let's choose 10% of our total users
#usuarios_al_azar = np.random.choice(datos['Identificador Socio'].unique(),
 #                               size=int(len(datos['Identificador Socio'].unique())*0.1),
  #                              replace=False)

#datos = datos.loc[datos['Identificador Socio'].isin(usuarios_al_azar)]

In [130]:
datos.head(2)

,Biblioteca de Prestamo,Meson de Circulación,Cod Biblioteca origen Item,Biblioteca origen Item,Cod de Localizacion,Localización Item,Prestamos en Sala,Prestamos Externo,Barcode,MMS Id,...,Fecha de ingreso del Item,Categoría Socio,Identificador Socio,dewey,Dewey_1,Dewey3,Dewey4.1,Dewey4.1 - Copia,Dewey_2,TIPO MATERIAL
0,Biblioteca Luis Ángel Arango,Préstamo Externo Calle 12 - Solo socios,BLAA,Biblioteca Luis Ángel Arango,DC1,Depósito C1,0,1,29004006168206,991009934269707486,...,1997-12-30 00:00:00,Categoría Biblioteca Virtual,3900488,Co868.5 S45p1,2,868.5 S45p1,868,868.0,860,LIBRO
1,Biblioteca Luis Ángel Arango,Préstamo Externo Calle 12 - Solo socios,BLAA,Biblioteca Luis Ángel Arango,DD1,Depósito D1,0,1,29004005863906,991013769509707486,...,1997-02-24 00:00:00,Categoría Biblioteca Virtual,3900488,928.61 S45j1,0,928.61 S45j1,928,928.0,920,LIBRO


In [155]:
from sklearn.preprocessing import LabelEncoder

# Crear los codificadores
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()
#author_encoder = LabelEncoder()
#category_encoder = LabelEncoder()

# Codificar usuarios y libros
datos['Identificador Socio'] = user_encoder.fit_transform(datos['Identificador Socio'])
datos['Título'] = item_encoder.fit_transform(datos['Título'])
#datos['Autor'] = author_encoder.fit_transform(datos['Autor'])
#datos['Categoría Socio'] = category_encoder.fit_transform(datos['Categoría Socio'])

# Los valores únicos de usuarios y libros
num_users = len(user_encoder.classes_)
num_items = len(item_encoder.classes_)
#num_authors = len(author_encoder.classes_)
#num_categories = len(category_encoder.classes_)

print(f"Total de usuarios: {num_users}")
print(f"Total de libros: {num_items}")
#print(f"Total de autores: {num_authors}")
#print(f"Total de categorías: {num_categories}")


Total de usuarios: 8136
Total de libros: 40111


In [156]:
datos['rank_recientes'] = datos.groupby(['Identificador Socio'])['Fecha de préstamo'].rank(method='first', ascending=False)

train = datos[datos['rank_recientes'] != 1]
test = datos[datos['rank_recientes'] == 1]

train = train[['Identificador Socio', 'MMS Id', 'Título']]
test = test[['Identificador Socio', 'MMS Id', 'Título']]

In [157]:
train.head()

,Identificador Socio,MMS Id,Título
0,4953,991009934269707486,30358
1,4953,991013769509707486,20795
2,4953,991008897989707486,4632
3,4953,991005656579707486,1288
6,4953,991003250799707486,33601


In [158]:
train.loc[train["Identificador Socio"]==83].head()

,Identificador Socio,MMS Id,Título
262639,83,991004270189707486,22634
264253,83,991012338009707486,22243
265621,83,991008347079707486,14152
265639,83,991005247319707486,25035
265677,83,991002129449707486,22677


In [159]:
test.loc[test["Identificador Socio"]==83].head()

,Identificador Socio,MMS Id,Título
265674,83,991005319229707486,1057


In [160]:
train.loc[:, 'MMS Id'] = 1

In [161]:
train["MMS Id"].unique()

array([1])

In [162]:
# Get a list of all movie IDs
all_movieIds = datos['Título'].unique()

# Placeholders that will hold the training data
users, items, labels = [], [], []

# This is the set of items that each user has interaction with
user_item_set = set(zip(train['Identificador Socio'], train['Título']))

# 4:1 ratio of negative to positive samples
num_negatives = 4

for (u, i) in user_item_set:
    users.append(u)
    items.append(i)
    labels.append(1) # items that the user has interacted with are positive
    for _ in range(num_negatives):
        # randomly select an item
        negative_item = np.random.choice(all_movieIds)
        # check that the user has not interacted with this item
        while (u, negative_item) in user_item_set:
            negative_item = np.random.choice(all_movieIds)
        users.append(u)
        items.append(negative_item)
        labels.append(0) # items not interacted with are negative

In [163]:
np.sum(labels)

163331

In [140]:
import torch
from torch.utils.data import Dataset

class MovieLensTrainDataset(Dataset):
    """MovieLens PyTorch Dataset for Training

    Args:
        ratings (pd.DataFrame): Dataframe containing the movie ratings
        all_movieIds (list): List containing all movieIds

    """

    def __init__(self,ratings,all_movieIds):
        self.users, self.items, self.labels = self.get_dataset(ratings, all_movieIds)

    def __len__(self):
        return len(self.users)

    def __getitem__(self, idx):
        return self.users[idx], self.items[idx], self.labels[idx]

    def get_dataset(self,ratings, all_movieIds):
        users, items, labels = [], [], []
        user_item_set = set(zip(ratings['Identificador Socio'], ratings['Título']))

        num_negatives = 4
        for u, i in user_item_set:
            users.append(u)
            items.append(i)
            labels.append(1)
            for _ in range(num_negatives):
                negative_item = np.random.choice(all_movieIds)
                while (u, negative_item) in user_item_set:
                    negative_item = np.random.choice(all_movieIds)
                users.append(u)
                items.append(negative_item)
                labels.append(0)

        return torch.tensor(users), torch.tensor(items), torch.tensor(labels)

In [141]:
class NCF(pl.LightningModule):
    """ Neural Collaborative Filtering (NCF)

        Args:
            num_users (int): Number of unique users
            num_items (int): Number of unique items or films
            ratings (pd.DataFrame): Dataframe containing the movie ratings for training
            all_movieIds (list): List containing all movieIds (train + test)
    """

    def __init__(self, num_users, num_items, ratings, all_movieIds):
        super().__init__()
        self.user_embedding = nn.Embedding(num_embeddings=num_users, embedding_dim=64)
        self.item_embedding = nn.Embedding(num_embeddings=num_items, embedding_dim=64)
        self.fc1 = nn.Linear(in_features=128, out_features=64)
        self.fc2 = nn.Linear(in_features=64, out_features=64)
        self.fc3 = nn.Linear(in_features=64, out_features=64)
        self.fc4 = nn.Linear(in_features=64, out_features=32)
        self.dropout = nn.Dropout(0.2)
        self.output = nn.Linear(in_features=32, out_features=1)
        self.ratings = ratings
        self.all_movieIds = all_movieIds

    def forward(self, user_input, item_input):

        # Pass through embedding layers
        user_embedded = self.user_embedding(user_input)
        item_embedded = self.item_embedding(item_input)

        # Concat the two embedding layers
        vector = torch.cat([user_embedded, item_embedded], dim=-1)

        # Pass through dense layer
        #vector = self.dropout(nn.ReLU()(self.fc1(vector)))
        #vector = self.dropout(nn.ReLU()(self.fc2(vector)))
        #vector = self.dropout(nn.ReLU()(self.fc3(vector)))
        #vector = nn.ReLU()(self.fc4(vector))

        vector = nn.ReLU()(self.fc1(vector))
        vector = self.dropout(vector)
        vector = nn.ReLU()(self.fc2(vector))
        vector = self.dropout(vector)
        vector = nn.ReLU()(self.fc3(vector))
        vector = self.dropout(vector)
        vector = nn.ReLU()(self.fc4(vector))
        vector = self.dropout(vector)


        # Output layer
        pred = nn.Sigmoid()(self.output(vector))

        return pred

    def training_step(self, batch, batch_idx):
        user_input, item_input, labels = batch
        predicted_labels = self(user_input, item_input)
        loss = nn.BCELoss()(predicted_labels, labels.view(-1, 1).float())
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.01)

    def train_dataloader(self):
        return DataLoader(MovieLensTrainDataset(self.ratings, self.all_movieIds),
                          batch_size=200, num_workers=4)

In [142]:
datos.head(3)

,Biblioteca de Prestamo,Meson de Circulación,Cod Biblioteca origen Item,Biblioteca origen Item,Cod de Localizacion,Localización Item,Prestamos en Sala,Prestamos Externo,Barcode,MMS Id,...,Categoría Socio,Identificador Socio,dewey,Dewey_1,Dewey3,Dewey4.1,Dewey4.1 - Copia,Dewey_2,TIPO MATERIAL,rank_recientes
0,Biblioteca Luis Ángel Arango,Préstamo Externo Calle 12 - Solo socios,BLAA,Biblioteca Luis Ángel Arango,DC1,Depósito C1,0,1,29004006168206,991009934269707486,...,Categoría Biblioteca Virtual,4953,Co868.5 S45p1,2,868.5 S45p1,868,868.0,860,LIBRO,16.0
1,Biblioteca Luis Ángel Arango,Préstamo Externo Calle 12 - Solo socios,BLAA,Biblioteca Luis Ángel Arango,DD1,Depósito D1,0,1,29004005863906,991013769509707486,...,Categoría Biblioteca Virtual,4953,928.61 S45j1,0,928.61 S45j1,928,928.0,920,LIBRO,29.0
2,Biblioteca Luis Ángel Arango,Préstamo Externo Calle 12 - Solo socios,BLAA,Biblioteca Luis Ángel Arango,DD1,Depósito D1,0,1,29004007799355,991008897989707486,...,Categoría Biblioteca Virtual,4953,986.251 B82,0,986.251 B82,986,986.0,980,LIBRO,33.0


In [143]:
num_users = train['Identificador Socio'].max()+1
num_items = train['Título'].max()+1
all_movieIds = train['Título'].unique()
print(num_users)
print(num_items)
print(len(all_movieIds))

8136
40111
39175


In [144]:
train["MMS Id"].unique()

array([1])

In [145]:
train = train.drop(columns=["MMS Id"])

In [146]:
train.head(3)

,Identificador Socio,Título
0,4953,30358
1,4953,20795
2,4953,4632


In [147]:
model = NCF(num_users, num_items, train, all_movieIds)

#trainer = pl.Trainer(max_epochs=5, gpus=1, reload_dataloaders_every_n_epochs=1,
                   #  enable_progress_bar=True)
trainer = pl.Trainer(max_epochs=1, accelerator='cpu', devices=1, reload_dataloaders_every_n_epochs=10, enable_progress_bar=True)
trainer.fit(model)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name           | Type      | Params
---------------------------------------------
0 | user_embedding | Embedding | 520 K 
1 | item_embedding | Embedding | 2.6 M 
2 | fc1            | Linear    | 8.3 K 
3 | fc2            | Linear    | 4.2 K 
4 | fc3            | Linear    | 4.2 K 
5 | fc4            | Linear    | 2.1 K 
6 | dropout        | Dropout   | 0     
7 | output         | Linear    | 33    
---------------------------------------------
3.1 M     Trainable params
0         Non-trainable params
3.1 M     Total params
12.426    Total estimated model params size (MB)
/usr/local/lib/python3.10/dist-packages/torch/utils

Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.


In [151]:
# User-item pairs for testing
test_user_item_set = set(zip(test['Identificador Socio'], test['Título']))

# Dict of all items that are interacted with by each user
user_interacted_items = datos.groupby('Identificador Socio')['Título'].apply(list).to_dict()

In [164]:
hits = []
for (u,i) in test_user_item_set:
    interacted_items = user_interacted_items[u]
    not_interacted_items = set(all_movieIds) - set(interacted_items)
    selected_not_interacted = list(np.random.choice(list(not_interacted_items), 99))
    test_items = selected_not_interacted + [i]

    predicted_labels = np.squeeze(model(torch.tensor([u]*100),
                                        torch.tensor(test_items)).detach().numpy())

    top20_items = [test_items[i] for i in np.argsort(predicted_labels)[::-1][0:20].tolist()]

    if i in top20_items:
        hits.append(1)
    else:
        hits.append(0)

print("The Hit Ratio @ 20 is {:.2f}".format(np.average(hits)))

The Hit Ratio @ 20 is 0.81


In [150]:
import random
import torch
import numpy as np

model.eval()

hits = []
for (u, i) in test_user_item_set:
    interacted_items = user_interacted_items[u]
    not_interacted_items = set(all_movieIds) - set(interacted_items)
    selected_not_interacted = random.sample(list(not_interacted_items), 99)
    test_items = selected_not_interacted + [i]

    user_batch = torch.tensor([u] * 100)
    item_batch = torch.tensor(test_items)
    predicted_labels = np.squeeze(model(user_batch, item_batch).detach().numpy())

    top20_items = [test_items[idx] for idx in np.argsort(predicted_labels)[::-1][:20]]

    hits.append(1 if i in top20_items else 0)

# Resultado final
print("The Hit Ratio @ 20 is {:.2%}".format(np.average(hits)))


The Hit Ratio @ 20 is 80.78%
